In [1]:
# part of scraping data from comments 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup as soup
import re
import os
import wget
import time
from splinter import Browser
from selenium.webdriver.chrome.options import Options

In [2]:
# Specify the path to the Chromedriver executable
executable_path = 'C:/Users/lejone/chromedriver.exe'

# Set Chrome options and include the executable path
chrome_options = Options()
chrome_options.executable_path = executable_path

# Create a browser instance with Chrome and the specified options
browser = Browser('chrome', options=chrome_options)

# Navigate to the Facebook login page
url = "https://www.facebook.com/"
browser.visit(url)

In [3]:
# Navigate to a specific Facebook page
# url = 'https://web.facebook.com/BritainsGotTalent?_rdc=1&_rdr'
# url = 'https://www.facebook.com/MejametalanaNewsroom'
# url = 'https://www.facebook.com/groups/585988568211873'
# url = 'https://www.facebook.com/profile.php?id=61551520045088'
url = 'https://web.facebook.com/VodacomLesotho'
# url = 'https://www.facebook.com/search/top?q=bosslogic'
# browser.visit(url)

# scroll down to load more results
scroll_count = 3
scroll_delay = 2

for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_delay)

# Parser the HTML
html = browser.html
# creating a BeautifulSoup object from the scrapped HTML
page_soup = soup(html, 'html.parser')

# check if HTML was scrapped correctly
# print(page_soup)

In [4]:
# extracting necessary data/info and insert into lists AND CLEANING
caption_box = page_soup.find_all('div', class_="x1iorvi4 x1pi30zi x1l90r2v x1swvt13")
caption_list = [caption.text.strip() for caption in caption_box]

name_box = page_soup.find_all('span', class_="xt0psk2")
name_list = [name.text.strip() for name in name_box]

comment_box = page_soup.find_all('div', class_="x1lliihq xjkvuk6 x1iorvi4")
comment_list = [comment.text.strip() for comment in comment_box]

# comment_list, caption_list, name_list
comment_list

import csv
# Open a CSV file in append mode
with open('output.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Iterate through the data list and write each row
    for comment in comment_list:
        writer.writerow([comment])

print("Data saved to output.csv")

Data saved to output.csv


In [5]:
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

df = pd.read_csv('output.csv')
df

,Comment
0,"('Gauteng re e thola joang rea kopa',)"
1,"('Hake lebetse password??',)"
2,"('Hakea staka',)"
3,"('',)"
4,Phoka
...,...
103,NaN
104,Congratulations Mme Mathabelo
105,NaN
106,NaN


In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Load the CSV file into a DataFrame
df = pd.read_csv('output.csv')

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for text preprocessing
def preprocess_text(text):
    # Check if the text is NaN
    if pd.isna(text): 
        # Return an empty string if NaN
        return ''  
    else:
        # Tokenize the text
        tokens = word_tokenize(text.lower())
        
        # Remove stopwords
        tokens = [token for token in tokens if token not in stopwords.words('english')]
        
        # Lemmatize tokens
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join tokens back into a cleaned text
        cleaned_text = ' '.join(tokens)
        
        return cleaned_text

# Apply the preprocessing function to the 'Comment' column and create a new column 'Cleaned_Comment'
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)

# Display the first or few few rows of the DataFrame to verify the data including the cleaned data
df.tail()

,Comment,Cleaned_Comment
103,NaN,
104,Congratulations Mme Mathabelo,congratulation mme mathabelo
105,NaN,
106,NaN,
107,NaN,


In [13]:

import pandas as pd
import joblib
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load your trained model (it was saved using joblib)
loaded_model = joblib.load('C:/Users/lejone/Desktop/New folder/kha90.joblib')  

# Load the dataset into a DataFrame ('output.csv' contains dataset of scrpped data)
df = pd.read_csv('output.csv')

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function for text preprocessing
def preprocess_text(text):
    # Check if the text is NaN
    if pd.isna(text): 
        # Return an empty string if NaN
        return ''  
    else:
        # Tokenize the text
        tokens = word_tokenize(text.lower())
        
        # Remove stopwords
        tokens = [token for token in tokens if token not in stopwords.words('english')]
        
        # Lemmatize tokens
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join tokens back into a cleaned text
        cleaned_text = ' '.join(tokens)
        
        return cleaned_text

# Apply the text preprocessing function to the 'Comment' column and create a new column 'Cleaned_Comment'
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)

# Use the trained model to predict the 'bullying_type' for each text
df['bullying_type'] = df['Cleaned_Comment'].apply(lambda x: loaded_model.predict([x])[0])

# Save the DataFrame with the predicted 'bullying_type' column to a new CSV file
df.to_csv('output_with_predictions.csv', index=False)
print("Well saved to csv")
# Display the DataFrame with the predicted 'bullying_type' column
df


Well saved to csv


,Comment,Cleaned_Comment,bullying_type
0,"('Gauteng re e thola joang rea kopa',)","( 'gauteng e thola joang rea kopa ' , )",bully
1,"('Hake lebetse password??',)","( 'hake lebetse password ? ? ' , )",bully
2,"('Hakea staka',)","( 'hakea staka ' , )",bully
3,"('',)","( `` , )",bully
4,Phoka,phoka,bully
...,...,...,...
80,"1. She scratches her nose, mouth, or ear more ...","1. scratch nose , mouth , ear frequently.2 . b...",bully
81,Ťőmåĺ Müsțąfā nice,ťőmåĺ müsțąfā nice,bully
82,Hao buwa ka monna ya ratang batho ebile aba h...,hao buwa ka monna ya ratang batho ebile aba hl...,non-bully
83,Themba Noseballkea leboha hlee my brother,themba noseballkea leboha hlee brother,bully
